# MLP for the Congressional Voting Dataset

## Importing the Data

In [1]:
import random

import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Load and preprocess dataset
df = pd.read_csv('all_datasets/CongressionalVotingID.shuf.lrn.csv')
df = df.drop(columns=['ID'])

In [2]:
label_encoder = LabelEncoder()
df["class"] = label_encoder.fit_transform(df["class"])

In [3]:
# Map vote values
vote_map = {"y": 1, "n": 0, "unknown": 0.5}
for col in df.columns[1:]:
    df[col] = df[col].map(vote_map)

X = df.drop(columns=["class"]).values.astype(np.float32)
y = df["class"].values.astype(np.int64)

In [4]:
# Define MLP model
class MLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
                nn.Linear(input_dim, 16),
                nn.ReLU(),
                nn.Linear(16, 8),
                nn.ReLU(),
                nn.Linear(8, 2),
        )

    def forward(self, x):
        return self.model(x)


split_ratios = [0.5, 0.3, 0.2, 0.1]

for split in split_ratios:
    print(f"\n--- Evaluating {int((1 - split) * 100)}/{int(split * 100)} Train/Test Split ---")

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split, random_state=42, stratify=y)

    X_train_tensor = torch.tensor(X_train)
    y_train_tensor = torch.tensor(y_train)
    X_test_tensor = torch.tensor(X_test)
    y_test_tensor = torch.tensor(y_test)

    model = MLP(X.shape[1])
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(1000):
        model.train()
        optimizer.zero_grad()
        logits = model(X_train_tensor)
        loss = loss_fn(logits, y_train_tensor)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        preds = torch.argmax(model(X_test_tensor), dim=1).numpy()
        y_true = y_test_tensor.numpy()

        acc = accuracy_score(y_true, preds)
        prec = precision_score(y_true, preds, zero_division=0)
        rec = recall_score(y_true, preds, zero_division=0)
        f1 = f1_score(y_true, preds, zero_division=0)

        print(f"Accuracy:  {acc:.4f}")
        print(f"Precision: {prec:.4f}")
        print(f"Recall:    {rec:.4f}")
        print(f"F1 Score:  {f1:.4f}")


--- Evaluating 50/50 Train/Test Split ---
Accuracy:  0.9358
Precision: 0.8800
Recall:    0.9778
F1 Score:  0.9263

--- Evaluating 70/30 Train/Test Split ---
Accuracy:  0.9848
Precision: 1.0000
Recall:    0.9643
F1 Score:  0.9818

--- Evaluating 80/20 Train/Test Split ---
Accuracy:  0.9773
Precision: 1.0000
Recall:    0.9444
F1 Score:  0.9714

--- Evaluating 90/10 Train/Test Split ---
Accuracy:  0.9545
Precision: 1.0000
Recall:    0.8889
F1 Score:  0.9412


## Testing out different MLPs

In [17]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import random

random.seed(42)
# Load and preprocess dataset
df = pd.read_csv('all_datasets/CongressionalVotingID.shuf.lrn.csv')
df = df.drop(columns=['ID'])

label_encoder = LabelEncoder()
df["class"] = label_encoder.fit_transform(df["class"])

# Map vote values
vote_map = {"y": 1, "n": 0, "unknown": 0.5}
for col in df.columns[1:]:
    df[col] = df[col].map(vote_map)

X = df.drop(columns=["class"]).values.astype(np.float32)
y = df["class"].values.astype(np.int64)

# Split once using 70/30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train)
X_test_tensor = torch.tensor(X_test)
y_test_tensor = torch.tensor(y_test)

# Define different model architectures
model_architectures = [
    nn.Sequential(
        nn.Linear(X.shape[1], 16),
        nn.ReLU(),
        nn.Linear(16, 8),
        nn.ReLU(),
        nn.Linear(8, 2),
    ),
    nn.Sequential(
        nn.Linear(X.shape[1], 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, 16),
        nn.ReLU(),
        nn.Linear(16, 2),
    ),
    nn.Sequential(
        nn.Linear(X.shape[1], 32),
        nn.Tanh(),
        nn.Linear(32, 16),
        nn.Tanh(),
        nn.Linear(16, 2),
    ),
    nn.Sequential(
        nn.Linear(X.shape[1], 128),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(64, 2),
    ),
]

# Store results
results = []

# Train and evaluate each model
for i, architecture in enumerate(model_architectures, 1):
    print(f"\n--- Evaluating Model Architecture {i} ---")


    class MLP(nn.Module):
        def __init__(self):
            super().__init__()
            self.model = architecture

        def forward(self, x):
            return self.model(x)


    model = MLP()
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(1000):
        model.train()
        optimizer.zero_grad()
        logits = model(X_train_tensor)
        loss = loss_fn(logits, y_train_tensor)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        preds = torch.argmax(model(X_test_tensor), dim=1).numpy()
        y_true = y_test_tensor.numpy()

        acc = accuracy_score(y_true, preds)
        prec = precision_score(y_true, preds, zero_division=0)
        rec = recall_score(y_true, preds, zero_division=0)
        f1 = f1_score(y_true, preds, zero_division=0)

        results.append((f"MLP{i}", acc, prec, rec, f1))

        # Print results in table format
print("\nMODEL\tAccuracy\tPrecision\tRecall\t\tF1-Score")
for model_name, acc, prec, rec, f1 in results:
    print(f"{model_name}\t{acc:.4f}\t\t{prec:.4f}\t\t{rec:.4f}\t\t{f1:.4f}")


--- Evaluating Model Architecture 1 ---

--- Evaluating Model Architecture 2 ---

--- Evaluating Model Architecture 3 ---

--- Evaluating Model Architecture 4 ---

MODEL	Accuracy	Precision	Recall		F1-Score
MLP1	0.9848		1.0000		0.9643		0.9818
MLP2	0.9848		1.0000		0.9643		0.9818
MLP3	0.9848		1.0000		0.9643		0.9818
MLP4	0.9697		0.9643		0.9643		0.9643


## Kaggle Code

In [14]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Load training data
df_train = pd.read_csv('all_datasets/CongressionalVotingID.shuf.lrn.csv')
df_train = df_train.drop(columns=['ID'])

label_encoder = LabelEncoder()
df_train["class"] = label_encoder.fit_transform(df_train["class"])

vote_map = {"y": 1, "n": 0, "unknown": 0.5}
for col in df_train.columns[1:]:
    df_train[col] = df_train[col].map(vote_map)

X_train = df_train.drop(columns=["class"]).values.astype(np.float32)
y_train = df_train["class"].values.astype(np.int64)

# Load test data
df_test = pd.read_csv('all_datasets/CongressionalVotingID.shuf.tes.csv')
test_ids = df_test["ID"].values
df_test = df_test.drop(columns=["ID"])

for col in df_test.columns:
    df_test[col] = df_test[col].map(vote_map)

X_test = df_test.values.astype(np.float32)


# Define MLP1
class MLP1(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(X.shape[1], 32),
            nn.Tanh(),
            nn.Linear(32, 16),
            nn.Tanh(),
            nn.Linear(16, 2),
        )

    def forward(self, x):
        return self.model(x)


# Train the model
X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train)
X_test_tensor = torch.tensor(X_test)

model = MLP1(X_train.shape[1])
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1000):
    model.train()
    optimizer.zero_grad()
    logits = model(X_train_tensor)
    loss = loss_fn(logits, y_train_tensor)
    loss.backward()
    optimizer.step()

# Predict on test set
model.eval()
with torch.no_grad():
    preds_tensor = torch.argmax(model(X_test_tensor), dim=1)
    preds = preds_tensor.numpy()

# Save submission v2: decoded predictions
decoded_preds = label_encoder.inverse_transform(preds)
df_submission_v2 = pd.DataFrame({"ID": test_ids, "class": decoded_preds})
df_submission_v2.to_csv("submission_mlp_v2.csv", index=False)

print("Submissions saved as 'submission_mlp_v2.csv'")

Submissions saved as 'submission_mlp_v2.csv'


## Cross Validation
using the best model from the previous section

In [19]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define MLP as a class for re-initialization
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(X.shape[1], 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 2)
        )

    def forward(self, x):
        return self.model(x)


# Set up
df = pd.read_csv('all_datasets/CongressionalVotingID.shuf.lrn.csv')
df = df.drop(columns=['ID'])

label_encoder = LabelEncoder()
df["class"] = label_encoder.fit_transform(df["class"])

# Map vote values
vote_map = {"y": 1, "n": 0, "unknown": 0.5}
for col in df.columns[1:]:
    df[col] = df[col].map(vote_map)

X = df.drop(columns=["class"]).values.astype(np.float32)
y = df["class"].values.astype(np.int64)
def run_cv(X, y, n_splits):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    all_results = []

    input_dim = X.shape[1]
    output_dim = len(np.unique(y))

    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y), 1):
        print(f"\n[Fold {fold}/{n_splits}]")

        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
        y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

        model = MLP(input_dim, output_dim).to(device)
        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

        # Training
        for epoch in range(200):
            model.train()
            optimizer.zero_grad()
            outputs = model(X_train_tensor)
            loss = loss_fn(outputs, y_train_tensor)
            loss.backward()
            optimizer.step()

        # Evaluation
        model.eval()
        with torch.no_grad():
            preds = model(X_test_tensor).argmax(dim=1).cpu().numpy()
            y_true = y_test_tensor.cpu().numpy()

            acc = accuracy_score(y_true, preds)
            prec = precision_score(y_true, preds, average='macro', zero_division=0)
            rec = recall_score(y_true, preds, average='macro', zero_division=0)
            f1 = f1_score(y_true, preds, average='macro', zero_division=0)

            all_results.append((acc, prec, rec, f1))

    accs, precs, recs, f1s = zip(*all_results)
    print(f"\n=== {n_splits}-Fold CV Results ===")
    print(f"Accuracy:  {np.mean(accs):.4f}")
    print(f"Precision: {np.mean(precs):.4f}")
    print(f"Recall:    {np.mean(recs):.4f}")
    print(f"F1 Score:  {np.mean(f1s):.4f}")
    print("=" * 30)

# Run 5-fold and 10-fold CV
run_cv(X, y, n_splits=5)
run_cv(X, y, n_splits=10)


[Fold 1/5]

[Fold 2/5]

[Fold 3/5]

[Fold 4/5]

[Fold 5/5]

=== 5-Fold CV Results ===
Accuracy:  0.9680
Precision: 0.9666
Recall:    0.9676
F1 Score:  0.9670

[Fold 1/10]

[Fold 2/10]

[Fold 3/10]

[Fold 4/10]

[Fold 5/10]

[Fold 6/10]

[Fold 7/10]

[Fold 8/10]

[Fold 9/10]

[Fold 10/10]

=== 10-Fold CV Results ===
Accuracy:  0.9634
Precision: 0.9651
Recall:    0.9621
F1 Score:  0.9623
